In [1]:
from datasets import load_dataset

ds = load_dataset("hf-audio/esb-datasets-test-only-sorted", "tedlium", split="test")

print(ds[0])

/home/ubuntu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'audio': {'path': None, 'array': array([ 0.02374268,  0.02404785,  0.0189209 , ..., -0.00292969,
       -0.00668335, -0.00897217]), 'sampling_rate': 16000}, 'dataset': 'tedlium', 'text': "making it can you see yes good this is actually me making the i'm not good at life face this is a piece of graffiti in my old neighborhood in berkeley california where i did my phd on why we're better in games than we are in real life and this is a problem that a lot of gamers have we feel that we are not as good in reality as we are in games and i don't mean just good as in successful although that's part of it we do achieve more in game worlds but i also mean good as in motivated to do something that matters inspired to collaborate and to cooperate and when we're in game worlds", 'id': 'JaneMcGonigal-196.922-229.47-<o,f0,female>', 'audio_length_s': 32.548}


In [1]:
from datasets import load_dataset
from transformers import VoxtralForConditionalGeneration, AutoProcessor
import torch

# --- Your dataset snippet ---
ds = load_dataset("hf-audio/esb-datasets-test-only-sorted", "tedlium", split="test")
sample = ds[0]  # {'dataset': 'tedlium', 'audio': {...}, 'id': ...}
audio_path = sample["audio"]["path"]  # local file path managed by `datasets`

# --- Voxtral setup ---
repo_id = "mistralai/Voxtral-Mini-3B-2507"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

processor = AutoProcessor.from_pretrained(repo_id)
model = VoxtralForConditionalGeneration.from_pretrained(repo_id, torch_dtype=dtype)
model.to(device)

# --- Build a transcription request and run generation (greedy) ---
# TED-LIUM is English; set language accordingly. Use the file path returned by HF datasets.
inputs = processor.apply_transcription_request(
    language="en",
    audio=audio_path,
    model_id=repo_id,  # lets the processor embed the right system tokens
)
inputs = inputs.to(device, dtype=dtype)

with torch.inference_mode():
    # Greedy decode for transcription-style determinism
    output_ids = model.generate(**inputs, max_new_tokens=400, do_sample=False)

# The model’s output includes the prompt tokens; slice them off before decoding.
pred = processor.batch_decode(
    output_ids[:, inputs.input_ids.shape[1]:],
    skip_special_tokens=True
)[0]

print("\n--- Voxtral transcription of ds[0] ---\n")
print(pred)


/home/ubuntu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.82it/s]


TypeError: 'NoneType' object is not iterable

In [3]:
from transformers import VoxtralForConditionalGeneration, AutoProcessor
import torch

device = "cuda"
repo_id = "mistralai/Voxtral-Mini-3B-2507"

ds = load_dataset("hf-audio/esb-datasets-test-only-sorted", "tedlium", split="test")
ex = ds[0]  # {'dataset': 'tedlium', 'audio': {...}, 'id': ...}
wave = ex["audio"]["array"]
sr   = int(ex["audio"]["sampling_rate"])  

processor = AutoProcessor.from_pretrained(repo_id)
model = VoxtralForConditionalGeneration.from_pretrained(repo_id, torch_dtype=torch.bfloat16, device_map=device)

inputs = processor.apply_transcription_request(
    language="en",
    audio=wave,       # pass numpy so soundfile can write it internally
    sampling_rate=sr,
    format=["WAV"],           # <-- important: list, and a valid container format
    model_id=repo_id,
    return_tensors="pt",      # required by this processor
)
inputs = inputs.to(device, dtype=torch.bfloat16)

outputs = model.generate(**inputs, max_new_tokens=500)
decoded_outputs = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("\nGenerated responses:")
print("=" * 80)
for decoded_output in decoded_outputs:
    print(decoded_output)
    print("=" * 80)

/home/ubuntu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.models.auto.processing_auto because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
# Install NeMo from trunk (as suggested in the model card)
# pip install "nemo_toolkit[asr,tts] @ git+https://github.com/NVIDIA/NeMo.git"

import torch
from nemo.collections.speechlm2.models import SALM

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Load the model
model = SALM.from_pretrained("nvidia/canary-qwen-2.5b")
model.to(device).eval()

# 2) Point to your audio file (16 kHz mono)
audio_file = "hello_world.wav"

# 3) Generate transcript (ASR mode)
with torch.inference_mode():
    out_ids = model.generate(
        prompts=[
            [
                {
                    "role": "user",
                    "content": f"Transcribe the following: {model.audio_locator_tag}",
                    "audio": [audio_file],
                }
            ]
        ],
        max_new_tokens=64,
    )

# 4) Decode and print
print(model.tokenizer.ids_to_text(out_ids[0].cpu()))


[NeMo I 2025-08-20 21:45:15 auto_tokenizer:234] 1 special tokens added, resize your model accordingly.
[NeMo I 2025-08-20 21:45:33 lora:30] LoRA adapter installed: LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='Qwen/Qwen3-1.7B', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=128, target_modules=['q_proj', 'v_proj'], lora_alpha=256, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))
[NeMo I 2025-08-20 21:45:33 features:306] PADDING: 0


AssertionError: Invalid value under prompt key 'audio': [ 0.02374268  0.02404785  0.0189209  ... -0.00292969 -0.00668335
 -0.00897217]

In [7]:
ex["audio"]

{'path': None,
 'array': array([ 0.02374268,  0.02404785,  0.0189209 , ..., -0.00292969,
        -0.00668335, -0.00897217]),
 'sampling_rate': 16000}

In [ ]:
from transformers import AutoProcessor

/home/ubuntu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
